# Projeto 1 - Ciência dos Dados

Nome: __Cauê Pileckas___

Nome: __Otávio Bianco___

Nome: __Victor Faria___

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
from unidecode import unidecode

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/cauedagostinho/Documents/Insper/2˚sem/Cdados/Projeto 1


### lista com palavras irrelevantes

In [3]:
lista_index_out = ["a", "o", "que", "do", "da", "de", "em", "um", "para", "e", "com", "nao", "uma", "os", "no", "se", "na", "por",
    "mais", "as", "dos", "como", "mas", "foi", "ao", "ele", "das", "tem", "à", "seu", "sua", "ou", "ser", "quando",
    "muito", "ha", "nos", "ja", "esta", "eu", "também", "so", "pelo", "pela", "ate", "isso", "ela", "entre", "era",
    "depois", "sem", "mesmo", "aos", "ter", "seus", "quem", "nas", "me", "esse", "eles", "estao", "voce", "tinha",
    "foram", "essa", "num", "nem", "suas", "meu", "as", "minha", "numa", "pelos", "elas", "havia", "seja",
    "qual", "sera", "nos", "tenho", "lhe", "deles", "essas", "esses", "pelas", "este", "fosse", "dele", "tu", "te",
    "voces", "vos", "lhes", "meus", "minhas", "teu", "tua", "teus", "tuas", "nosso", "nossa", "nossos", "nossas",
    "dela", "delas", "esta", "estes", "estas", "aquele", "aquela", "aqueles", "aquelas", "isto", "aquilo", "estou",
    "esta", "estamos", "estao", "estive", "esteve", "estivemos", "estiveram", "estava", "estavamos", "estavam",
    "estivera", "estiveramos", "esteja", "estejamos", "estejam", "estivesse", "estivessemos", "estivessem", "estiver",
    "estivermos", "estiverem", "hei", "ha", "havemos", "hao", "houve", "houvemos", "houveram", "houvera",
    "houveramos", "haja", "hajamos", "hajam", "houvesse", "houvessemos", "houvessem", "houver", "houvermos",
    "houverem", "houverei", "houverá", "houveremos", "houverão", "houveria", "houveríamos", "houveriam", "sou",
    "somos", "sao", "era", "eramos", "eram", "fui", "foi", "fomos", "foram", "fora", "foramos", "seja", "sejamos",
    "sejam", "fosse", "fossemos", "fossem", "for", "formos", "forem", "serei", "sera", "seremos", "serao", "seria",
    "seriamos", "seriam", "tenho", "temos", "tem", "tinha", "tinhamos", "tinham", "tive", "teve", "tivemos",
    "tiveram", "tivera", "tiveramos", "tenha", "tenhamos", "tenham", "tivesse", "tivessemos", "tivessem", "tiver",
    "tivermos", "tiverem", "terei", "tera", "teremos", "terao", "teria", "teriamos", "teriam"]

Carregando a base de dados com os tweets classificados manualmente:

In [4]:
train = pd.read_excel('data/dados_treino.xlsx')
train

,Mensagem,Target
0,"Pelo menos no Livro 1 - A guerra dos tronos, a...",Conteúdo
1,Não gostei. O livro o tempo todo traz narrativ...,Conteúdo
2,"Ok, tem partes picantes legais, mas grande par...",Conteúdo
3,A estoria de amor entre duas pessoas psicologi...,Conteúdo
4,Péssimo livro. Ao mesmo tempo que tenta vanglo...,Conteúdo
...,...,...
495,"CLARO QUE AS ESTRELAS NÃO SÃO PAR A OBRA, MAS ...",Outros
496,Tenho muitos livros sobre a Segunda Guerra mun...,Conteúdo
497,Na época que estourou eu paguei 105 na edição ...,Conteúdo
498,Livro apenas comenta sobre o que é leitura ráp...,Outros


In [5]:
test = pd.read_excel('data/dados_teste.xlsx')
test

,Mensagem,Target
0,Um livro sensacional. Tenho aplicado o conceit...,Outros
1,A edição traz uma introdução que é puro spoile...,Conteúdo
2,É tão ruim quanto o canal dele.,Outros
3,É um livro baseado no fator motivacional e em ...,Conteúdo
4,"Olar, Frequentemente tendo sua marca, merecida...",Outros
...,...,...
245,Muito fraco e passional. Visão distorcida de d...,Outros
246,Nas primeiras páginas já temos o propósito do ...,Conteúdo
247,Achei um livro muito superficial e enfadonho. ...,Outros
248,"A menos que queira uma versão POCKET, não comp...",Outros


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

##### CONTEÚDO:

Relevante: Tweets que abordam o enredo, personagens, temas, qualidade da escrita, análises críticas, recomendações, citações do livro, discussões sobre a trama, etc.
Não relevante: Tweets que não se relacionam ao conteúdo do livro, como anúncios de venda, spam, tweets promocionais não relacionados ao conteúdo, mensagens pessoais não relacionadas ao livro, etc.

##### PRODUTO:

Relevante: Tweets que falam sobre a qualidade física do livro, como a capa, papel, encadernação, impressão, ilustrações, edições especiais, brindes, etc.
Não relevante: Tweets que não se referem à qualidade física do livro, como tweets promocionais da editora, anúncios de livros em geral, avaliações de transporte/demora de entrega, mensagens pessoais não relacionadas à qualidade física do livro, etc.

##### OUTROS: 

Relevante: Tweets que abordam temas gerais, não relacionados ao livro, mas que podem ser relevantes para os leitores, como dicas de leitura, eventos literários, escritores famosos, etc.
Não relevante: Tweets que não têm relação com o livro ou com interesses literários, como discussões sobre filmes, esportes, política, entre outros assuntos que não têm relevância para os critérios definidos.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    unidecode_text = unidecode(text_subbed)
    return unidecode_text.lower()

def retorna_lista_mensagens(df, target):
    lista_palavras_conteudo = []
    palavras_conteudo = []


    for index, row in df.iterrows():
        if row['Target'] == target:
            lista_palavras_conteudo.append(row['Mensagem'].split())
        
    for lista in lista_palavras_conteudo:
        for i in range(len(lista)):
            palavras_conteudo.append(lista[i])
            
    return palavras_conteudo

def super_lista(df):
    string = []
    for mensagem in df['Mensagem']:
        lista_com_mensagem = mensagem.split()

        for palavra in lista_com_mensagem:
            if palavra in lista_index_out:
                lista_com_mensagem.remove(palavra)
                
            elif palavra[:4] == 'link' or palavra[:4] == 'http':
                lista_com_mensagem.remove(palavra)

        for word in lista_com_mensagem:
            string.append(word)

    super_string = string
    return super_string

In [7]:
train['Mensagem'] = train['Mensagem'].apply(cleanup)

In [8]:
# conteúdo -----
conteudo = train.loc[train['Target'] == 'Conteúdo']

# frequencia absoluta
conteudo_freq_abs = conteudo.value_counts()

# produto ------
produto = train.loc[train['Target'] == 'Produto']

# frequencia absoluta
produto_freq_abs = produto.value_counts()

# outros -------
outros = train.loc[train['Target'] == 'Outros']

# frequencia absoluta
outros_freq_abs = outros.value_counts()

### Tratando os dados em listas

In [9]:
# conteudo
str_conteudo = super_lista(conteudo)

# produto
str_produto = super_lista(produto)

# outros
str_outros = super_lista(outros)

### Juntando tudo para obter frequência relativa

In [10]:
palavras_juntas = str_conteudo + str_produto + str_outros

serie_palavras_juntas = pd.Series(palavras_juntas)

# frequencia relativa de todas as palavras
palavras_juntas_rel_frame = serie_palavras_juntas.value_counts(normalize=True)

# frequencia absoluta de todas as palavras
palavras_juntas_frame = serie_palavras_juntas.value_counts()

### Tirando repetições

In [11]:
palavras_sem_repeticoes = list(set(palavras_juntas))

In [12]:
serie_conteudo = pd.Series(str_conteudo)
serie_outros = pd.Series(str_outros)
serie_produto = pd.Series(str_produto)

# frequencia relativa por Target
df_conteudo_rel = serie_conteudo.value_counts(normalize=True)
df_outros_rel = serie_outros.value_counts(normalize=True)
df_produto_rel = serie_produto.value_counts(normalize=True)

# frequencia por "aparições"
df_conteudo_ap = serie_conteudo.value_counts()
df_outros_ap = serie_outros.value_counts()
df_produto_ap = serie_produto.value_counts()

### Calculando probabilidades por Target

In [13]:
prob_conteudo = len(serie_conteudo) / len(serie_palavras_juntas)
prob_outros = len(serie_outros) / len(serie_palavras_juntas)
prob_produto = len(serie_produto) / len(serie_palavras_juntas)

### Criando funções para o cálculo da probabilidade condicional

In [ ]:
# criação do suavizador para tratar a probabilidade + aquelas palavras que não tiverem no banco de dados
def Laplace(palavra, target, frequencia_absoluta, palavras_sem_repeticao):
    freq_total = 1
    resultado = 0
    
    if palavra in frequencia_absoluta:
        freq_abs = frequencia_absoluta[palavra]
    else:
        freq_abs = 0
    
    if target == 'Conteúdo':
        laplace = (freq_abs + freq_total) / (len(str_conteudo) + freq_total * len(palavras_sem_repeticoes))
        
    elif target == 'Outros':
        laplace = (freq_abs + freq_total) / (len(str_outros) + freq_total * len(palavras_sem_repeticoes))
        
    elif target == 'Produto':
        laplace = (freq_abs + freq_total) / (len(str_produto) + freq_total * len(palavras_sem_repeticoes))

        
    return laplace

def Condicional(target, feedback):
    
    if target == 'Conteúdo':
        prob_feedback_dado_conteudo = 1
        
        for palavra in feedback.split():
            prob_feedback_dado_conteudo *= Laplace(palavra, target, df_conteudo_ap, palavras_sem_repeticoes)
            
        prob_conteudo_dado_feedback = prob_conteudo * prob_feedback_dado_conteudo
        return prob_conteudo_dado_feedback
    
    elif target == 'Outros':
        prob_feedback_dado_outros = 1
        
        for palavra in feedback.split():
            prob_feedback_dado_outros *= Laplace(palavra, target, df_outros_ap, palavras_sem_repeticoes)
            
        prob_outros_dado_feedback = prob_outros * prob_feedback_dado_outros
        return prob_outros_dado_feedback
    
    elif target == 'Produto':
        prob_feedback_dado_produto = 1
        
        for palavra in feedback.split():
            prob_feedback_dado_produto *= Laplace(palavra, target, df_produto_ap, palavras_sem_repeticoes)
            
        prob_produto_dado_feedback = prob_produto * prob_feedback_dado_produto
        return prob_produto_dado_feedback
    
def Classificador(feedback):
    if Condicional('Conteúdo', feedback) == max([Condicional('Conteúdo', feedback), Condicional('Outros', feedback), Condicional('Produto', feedback)]):
        return 'Conteúdo'
    
    elif Condicional('Outros', feedback) == max([Condicional('Conteúdo', feedback), Condicional('Outros', feedback), Condicional('Produto', feedback)]):
        return 'Outros'
    
    elif Condicional('Produto', feedback) == max([Condicional('Conteúdo', feedback), Condicional('Outros', feedback), Condicional('Produto', feedback)]):
        return 'Produto'

# Aplicando análise em "Train"

In [26]:
freq_total = 1
train['Robot'] = None

for index, row in train.iterrows():

    prob_feedback_dado_conteudo = 1
    prob_feedback_dado_outros = 1
    prob_feedback_dado_produto = 1

    feedback = row['Mensagem']
    for palavra in feedback.split():

        # Fazendo Laplace com as palavras na mensagem
        # conteúdo ------
        if palavra in df_conteudo_ap:
            freq_abs_conteudo = df_conteudo_ap[palavra]

        else:
            freq_abs_conteudo = 0

        # outros ------
        if palavra in df_outros_ap:
            freq_abs_outros = df_outros_ap[palavra]

        else:
            freq_abs_outros = 0

        # produto -----
        if palavra in df_produto_ap:
            freq_abs_produto = df_produto_ap[palavra]

        else:
            freq_abs_produto = 0



        prob_feedback_dado_conteudo *= ((freq_abs_conteudo + freq_total) / (len(str_conteudo) + freq_total * len(palavras_sem_repeticoes)))
        prob_feedback_dado_outros *= ((freq_abs_outros + freq_total) / (len(str_outros) + freq_total * len(palavras_sem_repeticoes)))
        prob_feedback_dado_produto *= ((freq_abs_produto + freq_total) / (len(str_produto) + freq_total * len(palavras_sem_repeticoes)))

    prob_conteudo_dado_feedback = prob_conteudo * prob_feedback_dado_conteudo
    prob_outros_dado_feedback = prob_outros * prob_feedback_dado_outros
    prob_produto_dado_feedback = prob_produto * prob_feedback_dado_produto

    if prob_conteudo_dado_feedback == max([prob_conteudo_dado_feedback, prob_outros_dado_feedback, prob_produto_dado_feedback]):
        row['Robot'] = 'Conteúdo'

    elif prob_outros_dado_feedback == max([prob_conteudo_dado_feedback, prob_outros_dado_feedback, prob_produto_dado_feedback]):
        row['Robot'] = 'Outros'

    elif prob_produto_dado_feedback == max([prob_conteudo_dado_feedback, prob_outros_dado_feedback, prob_produto_dado_feedback]):
        row['Robot'] = 'Produto'

In [28]:
train

,Mensagem,Target,Robot
0,pelo menos no livro 1 a guerra dos tronos a t...,Conteúdo,Conteúdo
1,nao gostei o livro o tempo todo traz narrativa...,Conteúdo,Conteúdo
2,ok tem partes picantes legais mas grande parte...,Conteúdo,Conteúdo
3,a estoria de amor entre duas pessoas psicologi...,Conteúdo,Conteúdo
4,pessimo livro ao mesmo tempo que tenta vanglor...,Conteúdo,Conteúdo
...,...,...,...
495,claro que as estrelas nao sao par a obra mas p...,Outros,Outros
496,tenho muitos livros sobre a segunda guerra mun...,Conteúdo,Conteúdo
497,na epoca que estourou eu paguei 105 na edicao ...,Conteúdo,Conteúdo
498,livro apenas comenta sobre o que e leitura rap...,Outros,Outros


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

#### Verificando inicialmente com "crosstab"

In [29]:
#tabela = pd.crosstab(df_teste['Target'], df_teste['Robot'])
tabela = pd.crosstab(train['Robot'], train['Target'], normalize='index', margins=True)
tabela = tabela*100
tabela

Target,Conteúdo,Outros,Produto
Robot,,,
Conteúdo,74.509804,14.845938,10.644258
Outros,2.439024,91.056911,6.504065
Produto,0.000000,0.000000,100.000000
All,53.800000,33.000000,13.200000


#### Encontrando percentuais de Falsos Targets e Verdadeiros Targets

In [30]:
percent_verdadeiros_cont = len(train.loc[(train['Robot'] == 'Conteúdo') & (train['Target'] == 'Conteúdo')]) / len(train.loc[(train['Robot'] == 'Conteúdo')])
percent_falsos_cont = len(train.loc[(train['Robot'] == 'Conteúdo') & (train['Target'] != 'Conteúdo')]) / len(train.loc[(train['Robot'] == 'Conteúdo')])

percent_verdadeiros_prod = len(train.loc[(train['Robot'] == 'Produto') & (train['Target'] == 'Produto')]) / len(train.loc[(train['Robot'] == 'Produto')])
percent_falsos_prod = len(train.loc[(train['Robot'] == 'Produto') & (train['Target'] != 'Produto')]) / len(train.loc[(train['Robot'] == 'Produto')])

percent_verdadeiros_outros = len(train.loc[(train['Robot'] == 'Outros') & (train['Target'] == 'Outros')]) / len(train.loc[(train['Robot'] == 'Outros')])
percent_falsos_outros = len(train.loc[(train['Robot'] == 'Outros') & (train['Target'] != 'Outros')]) / len(train.loc[(train['Robot'] == 'Outros')])

print(f'Verdadeiros "Conteúdo": {percent_verdadeiros_cont*100:.2f}%  | Falsos "Conteúdo": {percent_falsos_cont*100:.2f}%')
print(f'Verdadeiros "Produto": {percent_verdadeiros_prod*100:.2f}%  | Falsos "Produto": {percent_falsos_prod*100:.2f}%')
print(f'Verdadeiros "Outros": {percent_verdadeiros_outros*100:.2f}%    | Falsos "Outros": {percent_falsos_outros*100:.2f}%')

Verdadeiros "Conteúdo": 74.51%  | Falsos "Conteúdo": 25.49%
Verdadeiros "Produto": 100.00%  | Falsos "Produto": 0.00%
Verdadeiros "Outros": 91.06%    | Falsos "Outros": 8.94%


#### Encontrando Acurácia do robô para "Train"

In [32]:
df_acuracia = train.copy()

In [34]:
df_acuracia.loc[df_acuracia['Target'] == df_acuracia['Robot'], 'Acuracia'] = 1
df_acuracia['Acuracia_sum'] = df_acuracia['Acuracia'].sum()

acuracia = (df_acuracia['Acuracia_sum']).values[-1]
acuracia_percent = (acuracia / len(df_acuracia)) * 100

print(f'Acurácia do robô: {acuracia} mensagens, equivalente a {acuracia_percent:.2f}% das mensagens')

Acurácia do robô: 398.0 mensagens, equivalente a 79.60% das mensagens


# Repetindo o processo para "Test"

### Aplicando análise...

In [37]:
freq_total = 1
test['Robot'] = None

for index, row in test.iterrows():

    prob_feedback_dado_conteudo = 1
    prob_feedback_dado_outros = 1
    prob_feedback_dado_produto = 1

    feedback = row['Mensagem']
    for palavra in feedback.split():

        # Fazendo Laplace com as palavras na mensagem
        # conteúdo ------
        if palavra in df_conteudo_ap:
            freq_abs_conteudo = df_conteudo_ap[palavra]

        else:
            freq_abs_conteudo = 0

        # outros ------
        if palavra in df_outros_ap:
            freq_abs_outros = df_outros_ap[palavra]

        else:
            freq_abs_outros = 0

        # produto -----
        if palavra in df_produto_ap:
            freq_abs_produto = df_produto_ap[palavra]

        else:
            freq_abs_produto = 0



        prob_feedback_dado_conteudo *= ((freq_abs_conteudo + freq_total) / (len(str_conteudo) + freq_total * len(palavras_sem_repeticoes)))
        prob_feedback_dado_outros *= ((freq_abs_outros + freq_total) / (len(str_outros) + freq_total * len(palavras_sem_repeticoes)))
        prob_feedback_dado_produto *= ((freq_abs_produto + freq_total) / (len(str_produto) + freq_total * len(palavras_sem_repeticoes)))

    prob_conteudo_dado_feedback = prob_conteudo * prob_feedback_dado_conteudo
    prob_outros_dado_feedback = prob_outros * prob_feedback_dado_outros
    prob_produto_dado_feedback = prob_produto * prob_feedback_dado_produto

    if prob_conteudo_dado_feedback == max([prob_conteudo_dado_feedback, prob_outros_dado_feedback, prob_produto_dado_feedback]):
        row['Robot'] = 'Conteúdo'

    elif prob_outros_dado_feedback == max([prob_conteudo_dado_feedback, prob_outros_dado_feedback, prob_produto_dado_feedback]):
        row['Robot'] = 'Outros'

    elif prob_produto_dado_feedback == max([prob_conteudo_dado_feedback, prob_outros_dado_feedback, prob_produto_dado_feedback]):
        row['Robot'] = 'Produto'

In [38]:
test

,Mensagem,Target,Robot
0,Um livro sensacional. Tenho aplicado o conceit...,Outros,Produto
1,A edição traz uma introdução que é puro spoile...,Conteúdo,Produto
2,É tão ruim quanto o canal dele.,Outros,Outros
3,É um livro baseado no fator motivacional e em ...,Conteúdo,Conteúdo
4,"Olar, Frequentemente tendo sua marca, merecida...",Outros,Conteúdo
...,...,...,...
245,Muito fraco e passional. Visão distorcida de d...,Outros,Produto
246,Nas primeiras páginas já temos o propósito do ...,Conteúdo,Produto
247,Achei um livro muito superficial e enfadonho. ...,Outros,Conteúdo
248,"A menos que queira uma versão POCKET, não comp...",Outros,Produto


In [39]:
tabela = pd.crosstab(test['Robot'], test['Target'], normalize='index', margins=True)
tabela = tabela*100
tabela

Target,Conteúdo,Outros,Produto
Robot,,,
Conteúdo,65.625000,26.562500,7.812500
Outros,36.666667,45.000000,18.333333
Produto,41.935484,33.870968,24.193548
All,52.800000,32.800000,14.400000


In [41]:
percent_verdadeiros_cont = len(test.loc[(test['Robot'] == 'Conteúdo') & (test['Target'] == 'Conteúdo')]) / len(test.loc[(test['Robot'] == 'Conteúdo')])
percent_falsos_cont = len(test.loc[(test['Robot'] == 'Conteúdo') & (test['Target'] != 'Conteúdo')]) / len(test.loc[(test['Robot'] == 'Conteúdo')])

percent_verdadeiros_prod = len(test.loc[(test['Robot'] == 'Produto') & (test['Target'] == 'Produto')]) / len(test.loc[(test['Robot'] == 'Produto')])
percent_falsos_prod = len(test.loc[(test['Robot'] == 'Produto') & (test['Target'] != 'Produto')]) / len(test.loc[(test['Robot'] == 'Produto')])

percent_verdadeiros_outros = len(test.loc[(test['Robot'] == 'Outros') & (test['Target'] == 'Outros')]) / len(test.loc[(test['Robot'] == 'Outros')])
percent_falsos_outros = len(test.loc[(test['Robot'] == 'Outros') & (test['Target'] != 'Outros')]) / len(test.loc[(test['Robot'] == 'Outros')])

print(f'Verdadeiros "Conteúdo": {percent_verdadeiros_cont*100:.2f}%  | Falsos "Conteúdo": {percent_falsos_cont*100:.2f}%')
print(f'Verdadeiros "Produto": {percent_verdadeiros_prod*100:.2f}%   | Falsos "Produto": {percent_falsos_prod*100:.2f}%')
print(f'Verdadeiros "Outros": {percent_verdadeiros_outros*100:.2f}%    | Falsos "Outros": {percent_falsos_outros*100:.2f}%')

Verdadeiros "Conteúdo": 65.62%  | Falsos "Conteúdo": 34.38%
Verdadeiros "Produto": 24.19%   | Falsos "Produto": 75.81%
Verdadeiros "Outros": 45.00%    | Falsos "Outros": 55.00%


In [42]:
df_acuracia = test.copy()

In [43]:
df_acuracia.loc[df_acuracia['Target'] == df_acuracia['Robot'], 'Acuracia'] = 1
df_acuracia['Acuracia_sum'] = df_acuracia['Acuracia'].sum()

acuracia = (df_acuracia['Acuracia_sum']).values[-1]
acuracia_percent = (acuracia / len(df_acuracia)) * 100

print(f'Acurácia do robô: {acuracia} mensagens, equivalente a {acuracia_percent:.2f}% das mensagens')

Acurácia do robô: 126.0 mensagens, equivalente a 50.40% das mensagens


___
### Concluindo

#### Respondendo questões:

1: Faça um comparativo qualitativo sobre os percentuais obtidos para que possa discutir a performance do seu classificador.

Resposta: O classificador teve uma melhor performance quando se trata de classificar as mensagens como "Conteúdos", isso se da pois no banco de dados "treino" a probabilidade da junção das palavras que formam alguma frase foi maior sempre que classificado como "Conteúdos".

2: 

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

In [ ]:
df_treino = train

for index, row in df_treino.iterrows():
    lista_feedback = (row['Mensagem'].split())
    
    for palavra in lista_feedback:
        if palavra in lista_index_out:
            lista_feedback.remove(palavra)
    
    feedback = " ".join(lista_feedback)
    
    row['Mensagem'] = feedback

In [ ]:
df_total = pd.concat([dataframe_teste, df_treino], ignore_index=True)

repeticoes = 100

for i in range(repeticoes):
    # Embaralhando as linhas do DataFrame
    df_embaralhado = df_total.sample(frac=1).reset_index(drop=True)
    
    # Divida o DataFrame em duas partes: 500 e 250 linhas
    parte1 = df_embaralhado[:500]
    parte2 = df_embaralhado[500:750]



___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**